In [1]:
import MDAnalysis as mda
from tripp import Trajectory
import numpy as np
import pandas as pds
import warnings
warnings.filterwarnings("ignore")

In [2]:
test = Trajectory('/Users/kafuman/Ivan/TrIPP/TrIPP/tutorial/files/1AKI_minstr.pdb',
                  '/Users/kafuman/Ivan/TrIPP/TrIPP/tutorial/files/1AKI_md2_300ns.xtc',
                  output_directory='testing',
                  output_prefix='test',
                  custom_terminal_oxygens=['OC1','OC2'])

In [6]:
df = pd.read_csv('testing/test_pka.csv',index_col='Time [ps]')
df['Trajectories'] = 'Unamed trajectory'


In [33]:
def make_pka_or_buriedness_df(file,trajectory_file):
    if isinstance(file, str):
        df = pd.read_csv(file, index_col='Time [ps]')
        df['Trajectories'] = 'Unnamed trajectory'

    elif isinstance(file, list):
        df_list = []
        for index, f in enumerate(file):
            df = pd.read_csv(f, index_col='Time [ps]')
            df['Trajectories'] = list(trajectory_file.keys())[index]
            df_list.append(df)
        df = pd.concat(df_list)

    elif file is None:
        df = None

    return df
trajectory_file = {'MD1':'tutorial/files/1AKI_md1_300ns.xtc',
                   'MD2':'tutorial/files/1AKI_md2_300ns.xtc'}
pka_df = make_pka_or_buriedness_df(['tutorial/output/1AKI_md1/1AKI_md1_pka.csv','tutorial/output/1AKI_md2/1AKI_md2_pka.csv'],trajectory_file)
buriedness_df = make_pka_or_buriedness_df(['tutorial/output/1AKI_md1/1AKI_md1_buriedness.csv','tutorial/output/1AKI_md2/1AKI_md2_buriedness.csv'],trajectory_file)

In [40]:
from tripp._create_clustering_matrix_ import *

In [82]:
from tripp._create_mda_universe_ import create_mda_universe

topology_file = 'tutorial/files/1AKI_minstr.pdb'
include_buriedness = False
residues = [35,52,114]
include_distances = False

trajectory_dict = {}
for key in trajectory_file.keys():
    traj = trajectory_file[key]
    trajectory_dict[key] = create_mda_universe(
        topology_file=topology_file, trajectory_file=traj
    )

for index, key in enumerate(trajectory_dict.keys()):
    df_traj_pka = pka_df[pka_df["Trajectories"] == key]
    if include_buriedness:
        df_traj_buriedness = buriedness_df[buriedness_df["Trajectories"] == key]
    elif not include_buriedness:
        df_traj_buriedness = None
    if index == 0:
        clustering_matrix, times, frames, trajectory_names = (
            extract_pka_distances_buriedness(key, trajectory_dict[key], df_traj_pka, df_traj_buriedness,residues, include_distances, include_buriedness)
        )

    else:
        partial_clustering_matrix, partial_times, partial_frames, partial_trajectory_names = extract_pka_distances_buriedness(key, trajectory_dict[key], 
                                                                                                                                df_traj_pka, 
                                                                                                                                df_traj_buriedness,
                                                                                                                                residues, 
                                                                                                                                include_distances, 
                                                                                                                                include_buriedness)
        clustering_matrix = np.concatenate((clustering_matrix, partial_clustering_matrix), axis=0)
        times = np.concatenate((times, partial_times), axis=0)
        frames = np.concatenate((frames, partial_frames), axis=0)
        trajectory_names = np.concatenate((trajectory_names, partial_trajectory_names), axis=0)

In [159]:
test.universe.select_atoms('((around 2 (resid 129 and backbone and type C and not name CA)) and resid 129) and type O or (resid 129 and backbone and type C and not name CA)').atoms.names

array(['C', 'OC1', 'OC2'], dtype=object)

In [63]:
clustering_matrix.shape

(6172, 9)

In [65]:
u = mda.Universe(topology_file,trajectory_file['MD1'])

In [69]:
u.select_atoms('resid 35').residues.resnames[0] + u.select_atoms('resid 35').residues.resids[0]

35

In [83]:
from sklearn_extra.cluster import KMedoids


In [84]:
clustering_matrix

array([[-0.37320718, -1.00642808,  1.24783831],
       [-0.38333872, -0.91271139,  1.24783831],
       [-0.4010689 , -0.97856636,  1.24023966],
       ...,
       [-0.51553341, -0.57628602,  1.29945071],
       [-0.63703862, -0.65728949,  1.27919984],
       [-0.88004905, -0.77879471,  1.51714755]])

In [85]:
clust = KMedoids(n_clusters=2,init='k-medoids++').fit(X=clustering_matrix)

In [79]:
clust.predict(clustering_matrix)

array([1, 1, 1, ..., 0, 0, 0])